In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from thermoengine import model, equilibrate, core
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
from contextlib import redirect_stdout
import io
_f = io.StringIO()
import timeout_decorator
from joblib import Parallel

In [2]:
modelDB = model.Database(liq_mod='pMELTS')

In [3]:
Liq = modelDB.get_phase('Liq')
Ol = modelDB.get_phase('Ol')
Cpx = modelDB.get_phase('Cpx')
Opx = modelDB.get_phase('Opx')
Fsp = modelDB.get_phase('Fsp')
Spn = modelDB.get_phase('SplS')
Grt = modelDB.get_phase('Grt')
Fo = modelDB.get_phase('Fo')

In [4]:
%cd working
from pyximport import install
install()
import Simple_Solution
%cd ..
db_ss = model.Database(database='CoderModule', calib=True,
                       phase_tuple=('Simple_Solution', {
                           'Ol':['Olivine','solution'],
                           'Fo':['Forsterite','pure'],
                           'Fa':['Fayalite','pure']}))

/Users/simonmatthews/Dropbox/projects/NewMeltingModel/working
/Users/simonmatthews/Dropbox/projects/NewMeltingModel


In [5]:
ol_simple = db_ss.get_phase('Ol')

Solution phase code generated by the coder module does not yet provide information on solution species. Species are proxied by components.
Solution phase code generated by the coder module does not yet provide information on species properties. Species are proxied by components.


In [6]:
elm_sys = ['H','O','Na','Mg','Al','Si','P','K','Ca','Ti','Cr','Mn','Fe','Co','Ni']
phs_sys = [Liq, Ol, Cpx, Opx, Fsp, Spn, Grt]
phs_sys_simple = [Liq, Cpx, Opx, Fsp, Spn, Grt, ol_simple]
phase_objs = {'liq': Liq, 'ol': Ol, 'cpx': Cpx, 'opx': Opx, 'pl': Fsp, 'spn': Spn, 'g': Grt}
phase_objs_simple = {'liq': Liq, 'ol': ol_simple, 'cpx': Cpx, 'opx': Opx, 'pl': Fsp, 'spn': Spn, 'g': Grt}

In [7]:
# KLB-1 Davis et al. (2009)
# lz_grm_oxides = {
#     'SiO2':  44.84, 
#     'TiO2':   0.11, 
#     'Al2O3':  3.51, 
#     'Fe2O3':  8.20*0.032/(55.845+16)*(55.845*2+16*3)/2,
#     'Cr2O3':  0.32, 
#     'FeO':    8.20*(1-0.032), 
#     'MnO':    0.12,
#     'MgO':   39.52, 
#     'NiO':    0.0, 
#     'CoO':    0.0,
#     'CaO':    3.07, 
#     'Na2O':   0.3, 
#     'K2O':    0.02, 
#     'P2O5':   0.0, 
#     'H2O':    0.0
# }

lz_grm_oxides ={'SiO2':  44.84, 
       'TiO2':   0.11, 
       'Al2O3':  3.51, 
       'Fe2O3':  8.20*0.032/(55.845+16)*(55.845*2+16*3)/2,
       # 'Cr2O3':  0.32, 
        'Cr2O3':  0.0, 
       'FeO':    8.20*(1-0.032), 
       # 'MnO':    0.12,
        'MnO': 0.0,
       'MgO':   39.52, 
       'NiO':    0.0, 
       'CoO':    0.0,
       'CaO':    3.07, 
       'Na2O':   0.3, 
       # 'K2O':    0.02, 
        'K2O': 0.0,
       'P2O5':   0.0, 
       # 'H2O':    0.0
       }

In [8]:
lz_mol_oxides = core.chem.format_mol_oxide_comp(lz_grm_oxides, convert_grams_to_moles=True)
lz_moles_end, lz_oxide_res = Liq.calc_endmember_comp(
    mol_oxide_comp=lz_mol_oxides, method='intrinsic', output_residual=True)
if not Liq.test_endmember_comp(lz_moles_end):
    print ("Calculated composition is infeasible!")
lz_mol_elm = Liq.convert_endmember_comp(lz_moles_end,output='moles_elements')

In [9]:
lz_blk_cmp = []
for elm in elm_sys:
    index = core.chem.PERIODIC_ORDER.tolist().index(elm)
    lz_blk_cmp.append(lz_mol_elm[index])
lz_blk_cmp = np.array(lz_blk_cmp)



In [10]:
phase_names = {'liq': 'Liquid', 'ol': 'Olivine', 'cpx': 'Clinopyroxene', 
               'opx': 'Orthopyroxene', 'pl': 'Feldspar', 'spn': 'Spinel', 'g': 'Garnet'}

In [11]:
@timeout_decorator.timeout(5)
def run_calc(t, p, bulk_comp, simple=False):
    with redirect_stdout(_f):
        if simple:
            equil = equilibrate.Equilibrate(elm_sys, phs_sys_simple)
        else:
            equil = equilibrate.Equilibrate(elm_sys, phs_sys)
        state = equil.execute(t, p, bulk_comp=bulk_comp, debug=0, stats=False)
    return state


In [25]:
p = np.arange(0.0, 80000.0, 500.0)
p[0] = 1.0
t = np.arange(1080.0, 2500.0, 10.0) + 273.15
t = t[::-1]

print(np.shape(p))
print(np.shape(t))

(160,)
(142,)


In [26]:
results = pd.DataFrame()

prev_f = 1.0

for k in range(np.shape(p)[0]):
    print(str(k+1) + ' of ' + str(np.shape(p)[0]))

    for j in range(np.shape(t)[0]):
        if prev_f > 0.005 or j == 0:
            try:
                state = run_calc(t[j], p[k], lz_blk_cmp)

                row = {}
                row['pressure'] = state.pressure / 1e4
                row['temperature'] = state.temperature - 273.15

                # Phase mass proportions
                total_mass = 0.0
                for ph in phase_names.keys():
                    row[ph+'_mass'] = state.tot_grams_phase(phase_names[ph])
                    total_mass += state.tot_grams_phase(phase_names[ph])

                for ph in phase_names.keys():
                    row[ph+'_mass'] = row[ph+'_mass'] / total_mass

                # Phase compositions
                for ph in phase_names.keys():
                    if ph == 'liq':
                        prefix = 'endm'
                    else:
                        prefix = ''
                    mol_endmembers = state.compositions(phase_names[ph], 'components')
                    component_names = phase_objs[ph].endmember_names
                    for i in range(len(mol_endmembers)):
                        row[ph + '_' + prefix + component_names[i]] = mol_endmembers[i]

                    oxide_comp = state.compositions(phase_names[ph], 'oxides', 'wt%')
                    for i in range(len(core.chem.OXIDE_ORDER)):
                        if core.chem.OXIDE_ORDER[i] in lz_grm_oxides.keys() and lz_grm_oxides[core.chem.OXIDE_ORDER[i]] > 0:
                            row[ph + '_' + core.chem.OXIDE_ORDER[i] + '_wtpt'] = oxide_comp[i]

                results = results.append(row, ignore_index=True)
                prev_f = row['liq_mass']
            except Exception:
                continue

            results.to_csv('results_backup.csv')

1 of 160
2 of 160


2022-03-28 13:42:31.240 python[21416:334566] Composition estimate is infeasible.
2022-03-28 13:42:31.240 python[21416:334566] species X0 chromite        = 0
2022-03-28 13:42:31.240 python[21416:334566] species X1 hercynite       = nan
2022-03-28 13:42:31.240 python[21416:334566] species X2 magnetite       = nan
2022-03-28 13:42:31.240 python[21416:334566] species X3 spinel          = nan
2022-03-28 13:42:31.240 python[21416:334566] species X4 ulvospinel      = nan
2022-03-28 13:42:31.240 python[21416:334566] species X5 magnesiochromit = 0
2022-03-28 13:42:31.240 python[21416:334566] species X6 magnesioferrite = nan
2022-03-28 13:42:31.240 python[21416:334566] species X7 qandilite       = nan
2022-03-28 13:42:31.240 python[21416:334566] chromite        X0 =             0, a =           nan, mu =           nan
2022-03-28 13:42:31.240 python[21416:334566] hercynite       X1 =           nan, a =           nan, mu =           nan
2022-03-28 13:42:31.240 python[21416:334566] magnetite       

3 of 160


2022-03-28 13:43:01.680 python[21416:334566] Composition estimate is infeasible.
2022-03-28 13:43:01.680 python[21416:334566] species X0 chromite        = 0
2022-03-28 13:43:01.681 python[21416:334566] species X1 hercynite       = nan
2022-03-28 13:43:01.681 python[21416:334566] species X2 magnetite       = nan
2022-03-28 13:43:01.681 python[21416:334566] species X3 spinel          = nan
2022-03-28 13:43:01.681 python[21416:334566] species X4 ulvospinel      = nan
2022-03-28 13:43:01.681 python[21416:334566] species X5 magnesiochromit = 0
2022-03-28 13:43:01.681 python[21416:334566] species X6 magnesioferrite = nan
2022-03-28 13:43:01.681 python[21416:334566] species X7 qandilite       = nan
2022-03-28 13:43:01.681 python[21416:334566] chromite        X0 =             0, a =           nan, mu =           nan
2022-03-28 13:43:01.681 python[21416:334566] hercynite       X1 =           nan, a =           nan, mu =           nan
2022-03-28 13:43:01.681 python[21416:334566] magnetite       

4 of 160
5 of 160


2022-03-28 13:44:07.011 python[21416:334566] Composition estimate is infeasible.
2022-03-28 13:44:07.011 python[21416:334566] species X0 chromite        = 0
2022-03-28 13:44:07.011 python[21416:334566] species X1 hercynite       = nan
2022-03-28 13:44:07.011 python[21416:334566] species X2 magnetite       = nan
2022-03-28 13:44:07.011 python[21416:334566] species X3 spinel          = nan
2022-03-28 13:44:07.011 python[21416:334566] species X4 ulvospinel      = nan
2022-03-28 13:44:07.011 python[21416:334566] species X5 magnesiochromit = 0
2022-03-28 13:44:07.011 python[21416:334566] species X6 magnesioferrite = nan
2022-03-28 13:44:07.011 python[21416:334566] species X7 qandilite       = nan
2022-03-28 13:44:07.012 python[21416:334566] chromite        X0 =             0, a =           nan, mu =           nan
2022-03-28 13:44:07.012 python[21416:334566] hercynite       X1 =           nan, a =           nan, mu =           nan
2022-03-28 13:44:07.012 python[21416:334566] magnetite       

6 of 160


2022-03-28 13:44:41.579 python[21416:334566] Composition estimate is infeasible.
2022-03-28 13:44:41.579 python[21416:334566] species X0 chromite        = 0
2022-03-28 13:44:41.579 python[21416:334566] species X1 hercynite       = nan
2022-03-28 13:44:41.579 python[21416:334566] species X2 magnetite       = nan
2022-03-28 13:44:41.579 python[21416:334566] species X3 spinel          = nan
2022-03-28 13:44:41.579 python[21416:334566] species X4 ulvospinel      = nan
2022-03-28 13:44:41.579 python[21416:334566] species X5 magnesiochromit = 0
2022-03-28 13:44:41.579 python[21416:334566] species X6 magnesioferrite = nan
2022-03-28 13:44:41.579 python[21416:334566] species X7 qandilite       = nan
2022-03-28 13:44:41.579 python[21416:334566] chromite        X0 =             0, a =           nan, mu =           nan
2022-03-28 13:44:41.579 python[21416:334566] hercynite       X1 =           nan, a =           nan, mu =           nan
2022-03-28 13:44:41.579 python[21416:334566] magnetite       

7 of 160
8 of 160
9 of 160
10 of 160
11 of 160
12 of 160
13 of 160
14 of 160
15 of 160
16 of 160
17 of 160
18 of 160
19 of 160
20 of 160
21 of 160
22 of 160
23 of 160
24 of 160
25 of 160
26 of 160
27 of 160
28 of 160
29 of 160
30 of 160
31 of 160
32 of 160
33 of 160
34 of 160
35 of 160
36 of 160
37 of 160
38 of 160
39 of 160
40 of 160
41 of 160
42 of 160
43 of 160
44 of 160
45 of 160
46 of 160
47 of 160
48 of 160
49 of 160
50 of 160
51 of 160
52 of 160
53 of 160
54 of 160
55 of 160
56 of 160
57 of 160
58 of 160
59 of 160
60 of 160
61 of 160
62 of 160
63 of 160
64 of 160
65 of 160
66 of 160
67 of 160
68 of 160
69 of 160
70 of 160
71 of 160
72 of 160
73 of 160
74 of 160
75 of 160
76 of 160
77 of 160
78 of 160
79 of 160
80 of 160
81 of 160
82 of 160
83 of 160
84 of 160
85 of 160
86 of 160
87 of 160
88 of 160
89 of 160
90 of 160
91 of 160
92 of 160
93 of 160
94 of 160
95 of 160
96 of 160
97 of 160
98 of 160
99 of 160
100 of 160
101 of 160
102 of 160
103 of 160
104 of 160
105 of 160
106 of 

In [27]:
results.to_csv('klb1_pmelts_grid.csv')

NEED TO WRITE CODE TO REMOVE DUPLICATE ROWS

In [45]:
state.print_state()

 
T =    1565.00 °C, P =      375.0 MPa
Liquid          moles:   0.641282 grams: 100.039
          Si4O8 form:  Si4O8          X:  0.0550  wt%    SiO2   44.82
           TiO2 form:  TiO2           X:  0.0021  wt%    TiO2    0.11
          Al4O6 form:  Al4O6          X:  0.0267  wt%   Al2O3    3.51
          Fe2O3 form:  Fe2O3          X:  0.0028  wt%   Fe2O3    0.29
        MgCr2O4 form:  MgCr2O4        X:  0.0033  wt%   Cr2O3    0.32
        Fe2SiO4 form:  Fe2SiO4        X:  0.0861  wt%     FeO    7.93
      MnSi0.5O2 form:  MnSi0.5O2      X:  0.0026  wt%     MnO    0.12
        Mg2SiO4 form:  Mg2SiO4        X:  0.7629  wt%     MgO   39.50
      NiSi0.5O2 form:  NiSi0.5O2      X:  0.0000  wt%     CaO    3.07
      CoSi0.5O2 form:  CoSi0.5O2      X:  0.0000  wt%    Na2O    0.30
       Ca2Si2O6 form:  Ca2Si2O6       X:  0.0427  wt%     K2O    0.02
    NaSi0.5O1.5 form:  NaSi0.5O1.     X:  0.0151  
        KAlSiO4 form:  KAlSiO4        X:  0.0007  
      Ca3(PO4)2 form:  Ca3(PO4)2      X

In [125]:
df = pd.DataFrame()

In [127]:
df.append(row, ignore_index=True)

,pressure,temperature,liq_mass,ol_mass,cpx_mass,opx_mass,pl_mass,spn_mass,g_mass,liq_endmSi4O8,...,g_TiO2_wtpt,g_Al2O3_wtpt,g_Fe2O3_wtpt,g_Cr2O3_wtpt,g_FeO_wtpt,g_MnO_wtpt,g_MgO_wtpt,g_CaO_wtpt,g_Na2O_wtpt,g_K2O_wtpt
0,1.0,1300.0,0.037941,0.600267,0.130033,0.209376,0.0,0.022383,0.0,0.004464,...,0.316292,6.172754,0.717391,0.0,3.946233,0.0,19.160831,16.880894,0.633931,0.0


In [9]:
equil = equilibrate.Equilibrate(elm_sys, phs_sys)

In [77]:
p = 1.0
t = 1535 + 273.15
# t = 1753.15

# with redirect_stdout(_f):

@timeout_decorator.timeout(10)
def run_calc():
    with redirect_stdout(_f):
        state = equil.execute(t, p, bulk_comp=lz_blk_cmp, debug=0, stats=True)
    return state

try:
    run_calc()
except Exception:
    print('timed out')

timed out


In [23]:
# lz_grm_oxides = {
#     'SiO2':  44.84, 
#     'TiO2':   0.11, 
#     'Al2O3':  3.51, 
#     'Fe2O3':  8.20*0.032/(55.845+16)*(55.845*2+16*3)/2,
#     'Cr2O3':  0.32, 
#     'FeO':    8.20*(1-0.032), 
#     'MnO':    0.12,
#     'MgO':   39.52, 
#     'NiO':    0.001, 
#     'CoO':    0.001,
#     'CaO':    3.07, 
#     'Na2O':   0.3, 
#     'K2O':    0.02, 
#     'P2O5':   0.001, 
#     'H2O':   0.01}

# lz_mol_oxides = core.chem.format_mol_oxide_comp(lz_grm_oxides, convert_grams_to_moles=True)
# lz_moles_end, lz_oxide_res = Liq.calc_endmember_comp(
#     mol_oxide_comp=lz_mol_oxides, method='intrinsic', output_residual=True)
# if not Liq.test_endmember_comp(lz_moles_end):
#     print ("Calculated composition is infeasible!")
# lz_mol_elm = Liq.convert_endmember_comp(lz_moles_end,output='moles_elements')
    
# lz_blk_cmp = []
# for elm in elm_sys:
#     index = core.chem.PERIODIC_ORDER.tolist().index(elm)
#     lz_blk_cmp.append(lz_mol_elm[index])
# lz_blk_cmp = np.array(lz_blk_cmp)

    
    
p = 0.2
t = 1520 + 273.15
equil = equilibrate.Equilibrate(elm_sys, phs_sys)
state = equil.execute(t, p, bulk_comp=lz_blk_cmp, debug=0, stats=True)

Add: Olivine
Quad (000) norm:  1.4078885908967e-01 Lin (016) step:  8.3473523778331e-01 func: -1.7453100628750e+06
Quad (001) norm:  1.2483542149824e-03 Lin (022) step:  1.0541176910856e+00 func: -1.7453103503859e+06
Quad (002) norm:  5.4921168111496e-04 Lin (020) step:  9.9915761485497e-01 func: -1.7453103510256e+06
Quad (003) norm:  4.3615277892110e-08 Lin (038) step:  1.0303868652312e+00 func: -1.7453103510256e+06
Quad (004) norm:  1.3253792022933e-09 Lin (038) step: -8.2184244492590e-01 func: -1.7453103510256e+06
Quad (005) norm:  2.4146865554566e-09 Lin (021) step:  7.0911012968913e-01 func: -1.7453103510256e+06
Minimal energy termination of quadratic loop.



In [24]:
state.print_state()

 
T =    1520.00 °C, P =        0.0 MPa
Liquid          moles:   0.549107 grams:  86.731
          Si4O8 form:  Si4O8          X:  0.0645  wt%    SiO2   45.51
           TiO2 form:  TiO2           X:  0.0025  wt%    TiO2    0.13
          Al4O6 form:  Al4O6          X:  0.0313  wt%   Al2O3    4.05
          Fe2O3 form:  Fe2O3          X:  0.0033  wt%   Fe2O3    0.34
        MgCr2O4 form:  MgCr2O4        X:  0.0000  wt%     FeO    8.45
        Fe2SiO4 form:  Fe2SiO4        X:  0.0929  wt%     MgO   37.71
      MnSi0.5O2 form:  MnSi0.5O2      X:  0.0000  wt%     CaO    3.47
        Mg2SiO4 form:  Mg2SiO4        X:  0.7389  wt%    Na2O    0.35
      NiSi0.5O2 form:  NiSi0.5O2      X:  0.0000  
      CoSi0.5O2 form:  CoSi0.5O2      X:  0.0000  
       Ca2Si2O6 form:  Ca2Si2O6       X:  0.0489  
    NaSi0.5O1.5 form:  NaSi0.5O1.     X:  0.0176  
        KAlSiO4 form:  KAlSiO4        X:  0.0000  
      Ca3(PO4)2 form:  Ca3(PO4)2      X:  0.0000  
            H2O form:  H2O            X:  0.0

In [122]:
equilibrate.Equilibrate?

Init signature: equilibrate.Equilibrate(element_l=None, phase_l=None, lagrange_l=None)
Docstring:     
Class for minimizing a generic thermodynamic potential in order to
calculate an equilibrium phase assemblage.

The default potential is the Gibbs free energy.

Parameters
----------
element_l : [], default None
    See documentation for element_list attribute.
phase_l : [], default None
    See documentation for phase_list attribute.
lagrange_l : [], default None
    See documentation for lagrange_list attribute.

Attributes
----------
A_omni_inv
bulk_comp
CTf
element_list
entropy
eps_linear
eps_minimal_energy
eps_quad_optimal
eps_quad_suboptimal
eps_rank
equil_cycle_max
equil_linear_min
lagrange_list
lagrange_moles
lagrange_no_mol_deriv
lagrange_use_omni
max_linear_iters
max_quad_iters
moles_in
moles_out
phase_list
phase_separ_threshold
reactions
rotate_orthog_proj
use_numpy_lstsq
volume
VT_null

Notes
-----
Alternate potentials are specified by stipulating appropriate Lagrange
trans

In [57]:
row = {}
row['pressure'] = state.pressure / 1e4
row['temperature'] = state.temperature - 273.15

# Phase mass proportions
total_mass = 0.0
for ph in phase_names.keys():
    row[ph+'_mass'] = state.tot_grams_phase(phase_names[ph])
    total_mass += state.tot_grams_phase(phase_names[ph])

for ph in phase_names.keys():
    row[ph+'_mass'] = row[ph+'_mass'] / total_mass

# Phase compositions
for ph in phase_names.keys():
    if ph == 'liq':
        prefix = 'endm'
    else:
        prefix = ''
    mol_endmembers = state.compositions(phase_names[ph], 'components')
    component_names = phase_objs[ph].endmember_names
    for i in range(len(mol_endmembers)):
        row[ph + '_' + prefix + component_names[i]] = mol_endmembers[i]
    
    oxide_comp = state.compositions('Clinopyroxene', 'oxides', 'wt%')
    for i in range(len(core.chem.OXIDE_ORDER)):
        if core.chem.OXIDE_ORDER[i] in lz_grm_oxides.keys() and lz_grm_oxides[core.chem.OXIDE_ORDER[i]] > 0:
            row[ph + '_' + core.chem.OXIDE_ORDER[i] + '_wtpt'] = oxide_comp[i]

In [58]:
row

{'pressure': 0.0001,
 'temperature': 2495.0,
 'liq_mass': 1.0,
 'ol_mass': 0.0,
 'cpx_mass': 0.0,
 'opx_mass': 0.0,
 'pl_mass': 0.0,
 'spn_mass': 0.0,
 'g_mass': 0.0,
 'liq_endmSi4O8': 0.03524188582470016,
 'liq_endmTiO2': 0.0013767415780962919,
 'liq_endmAl4O6': 0.017106254024503376,
 'liq_endmFe2O3': 0.0018261283669411127,
 'liq_endmMgCr2O4': 0.002105398900718599,
 'liq_endmFe2SiO4': 0.05524006769998191,
 'liq_endmMnSi0.5O2': 0.0016916323406270882,
 'liq_endmMg2SiO4': 0.48921635306003664,
 'liq_endmNiSi0.5O2': 0.0,
 'liq_endmCoSi0.5O2': 0.0,
 'liq_endmCa2Si2O6': 0.02737190483493047,
 'liq_endmNaSi0.5O1.5': 0.00968070767263848,
 'liq_endmKAlSiO4': 0.0004246491866906452,
 'liq_endmCa3(PO4)2': 0.0,
 'liq_endmH2O': 0.0,
 'liq_SiO2_wtpt': 0.0,
 'liq_TiO2_wtpt': 0.0,
 'liq_Al2O3_wtpt': 0.0,
 'liq_Fe2O3_wtpt': 0.0,
 'liq_Cr2O3_wtpt': 0.0,
 'liq_FeO_wtpt': 0.0,
 'liq_MnO_wtpt': 0.0,
 'liq_MgO_wtpt': 0.0,
 'liq_CaO_wtpt': 0.0,
 'liq_Na2O_wtpt': 0.0,
 'liq_K2O_wtpt': 0.0,
 'ol_tephroite': 0.0,

In [67]:
state.compositions('Clinopyroxene', 'oxides','wt%')

array([52.17167338,  0.3162922 ,  6.17275403,  0.71739121,  0.        ,
        3.9462327 ,  0.        , 19.16083121,  0.        ,  0.        ,
       16.88089398,  0.63393129,  0.        ,  0.        ,  0.        ,
        0.        ])

In [65]:
state.compositions?

Signature: state.compositions(phase_name=None, ctype='components', units='moles')
Docstring:
Returns compositions of phases in the system

Parameters
----------
phase_name : str, default None
    Name of phase in system
ctype : str, default 'components'
    Compositional descriptor. Permitted: 'components', 'oxides',
    'elements'
units : str, default 'moles'
    Units of composition. Permitted: 'moles', 'mole_frac', 'wt%'

Returns
-------
result : numpy array
    One dimensional vector

Notes
-----
If units is set to 'oxides' or 'elements', results are returned in an
array of standard length and order.
File:      ~/opt/anaconda3/lib/python3.7/site-packages/thermoengine/equilibrate.py
Type:      method
